# Spark SQL Exercise

Let's work with some airline data and play around/do some EDA.

In [1]:
import pyspark
from urllib import request

spark = pyspark.sql.SparkSession.builder.getOrCreate()

airports_path = 'data/airline/airport_codes.txt'


These are premade files collected from [Airline On-Time Performance and Causes of Flight Delays: On Time Data](https://catalog.data.gov/dataset/airline-on-time-performance-and-causes-of-flight-delays-on-time-data). The flight delay is missing the carrier id (i.e. which airline operated the plane)

You can generate your own data set with custom ranges for dates, and what data to include by selecting the appopriate boxes [here](https://transtats.bts.gov/DL_SelectFields.asp)

Let's load the data from the correct path and also make it into a view that SparkSQL and query.

In [4]:
airports = spark.read.csv(airports_path, header='true', inferSchema='true',
                          sep='\t')
airports.createOrReplaceTempView("airports")


In [11]:
flight_delay_url = 'https://s3.amazonaws.com/gamma-datasets/departuredelays.csv'

In [9]:
# Obtain Airports dataset
airports = spark.read.csv(airports_path, header='true', inferSchema='true',
                          sep='\t')
airports.createOrReplaceTempView("airports")

# Use urllib.request to download file
flight_delay_file, _ = request.urlretrieve(flight_delay_url)

# Obtain Departure Delays dataset
flightPerf = spark.read.csv(flight_delay_file, header='true')
flightPerf.createOrReplaceTempView("FlightPerformance")

In [10]:
flightPerf.printSchema()

root
 |-- date: string (nullable = true)
 |-- delay: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [ ]:
airports.printSchema()

In [12]:
spark.sql('SELECT CITY, IATA FROM airports WHERE state="IL" ').show()

+-----------+----+
|       CITY|IATA|
+-----------+----+
|Bloomington| BMI|
|  Champaign| CMI|
|    Chicago| MDW|
|    Chicago| ORD|
|    Chicago| CHI|
|    Decatur| DEC|
|     Marion| MWA|
|     Moline| MLI|
|     Peoria| PIA|
|     Quincy| UIN|
|   Rockford| RFD|
|Springfield| SPI|
+-----------+----+



In [13]:
spark.sql('SELECT * FROM FlightPerformance').show(5)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 5 rows



In [14]:
query = '''
SELECT destination, avg(distance) as distance, avg(delay) as avg_delay FROM FlightPerformance  
   WHERE origin="ORD" AND delay > 0
   GROUP BY destination
   ORDER BY avg_delay DESC
'''

spark.sql(query).show()

+-----------+--------+------------------+
|destination|distance|         avg_delay|
+-----------+--------+------------------+
|        RAP|   677.0|             121.5|
|        CHO|   492.0|             111.5|
|        ROA|   461.0| 70.22222222222223|
|        GSO|   512.0| 61.91208791208791|
|        CAE|   579.0| 61.81884057971015|
|        MOB|   677.0|61.093023255813954|
|        GSP|   502.0|61.054545454545455|
|        HSV|   443.0| 57.91818181818182|
|        MQT|   263.0| 56.04761904761905|
|        HNL|  3687.0|53.861111111111114|
|        SCE|   459.0| 53.73831775700935|
|        PNS|   690.0|53.642857142857146|
|        ELP|  1074.0|            53.625|
|        PWM|   782.0| 52.23809523809524|
|        ORF|   623.0|51.445945945945944|
|        FOE|   399.0| 51.32608695652174|
|        BOI|  1249.0|51.108108108108105|
|        MTJ|   939.0|50.074074074074076|
|        TYS|   413.0| 49.82175226586103|
|        BTV|   663.0| 49.41860465116279|
+-----------+--------+------------

## Exercise 1: 

What is the average delay for flights coming _into_ ORD?

In [ ]:
spark.sql('select avg()')

# Exercise 2:
What is the average delay for flights leaving from any airport in Illinois, broken up by destination?